In [ ]:
import data_modules
import torch
from data_modules.baseline_finetune_dm import BaselineFinetuneDM
from data_modules.generic_finetune_dm import GenericFinetuneDM

dm = GenericFinetuneDM(train_dir="data/topex-printer/train/", test_dir="data/topex-printer/test/")
# gftdm = GenericFinetuneDM()
dm.prepare_data()
dm.setup()
train_dataloader = dm.train_dataloader()
val_dataloader = dm.val_dataloader()
test_dataloader = dm.test_dataloader()

inputs, labels = next(iter(train_dataloader))
print(dm.num_classes)
print(len(dm.test.classes))
print(dm.test.classes == dm.num_classes)

In [ ]:
import os
import shutil

import pandas as pd

data = "/home/dennis/projects/evaluation-pipeline/data/datasets/visda2017_meshes/train"
out = "/home/dennis/projects/evaluation-pipeline/data/datasets/visda2017_meshgrid"

d = {}
for path, dns, fns in os.walk(data):
    for fn in fns:
        fnsplit = fn.split("__")
        mesh = fnsplit[0]
        angles = fnsplit[1].split("_")
        light_angle = angles[1]
        if f"{mesh}_{light_angle}" in d.keys():
            d[f"{mesh}_{light_angle}"].append(fn)
        else:
            d[f"{mesh}_{light_angle}"] = [fn]

In [ ]:
import os
import random

import torchvision
from PIL import Image
from torchvision import transforms as transforms
from torchvision.io import read_image
from torchvision.utils import make_grid


def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result


img_dir = "../data/modelnet10_toy/train/chair/chair_0001"

grid_size = (4, 4)
padding = 0
margin = 16
n_img = grid_size[0] * grid_size[1]
renders = os.listdir(img_dir)
# rnd_model = random.sample(renders, k=1)[0]
# rnd_model = rnd_model.split("_")[3]
# renders = [render for render in renders if render.split("_")[3] == rnd_model]
rnd_imgs = random.sample(renders, k=n_img)
imgs = [read_image(f"{img_dir}/{img_fn}") for img_fn in rnd_imgs]
grid = make_grid(imgs, nrow=grid_size[1], padding=padding)
img_grid = torchvision.transforms.ToPILImage()(grid)
img_grid = img_grid.resize((224, 224))
img_grid = add_margin(img_grid, margin, margin, margin, margin, (0, 0, 0))
img_grid

In [ ]:
import os

import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image

# Define the mean and standard deviation values from the ImageNet dataset
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]
# mean = [0.485, 0.456, 0.406]
# std = [0.229, 0.224, 0.225]

# Define a transform pipeline that applies any necessary image transformations and normalization
transform = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.RandomCrop((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ]
)

# Load an image from file (in this case, a JPEG file)
dir = "../data/visda2017/train"
for path, dir, fn in os.walk(dir):
    for f in fn:
        image = Image.open(f"{path}/{f}")
        print(f"minmax original: {np.array(image).min()}, {np.array(image).max()}")
        # Apply the transform pipeline to the image
        transformed_image = transform(image)
        # Print the minimum and maximum pixel values in the transformed image tensor
        print(f"minmax transformed: {transformed_image.min()}, {transformed_image.max()}")

In [ ]:
import torch
from torchvision import transforms

from src.datamodules.generic_finetune_dm import GenericFinetuneDM

dm = GenericFinetuneDM(
    train_dir="data/visda2017/train/",
    test_dir="data/visda2017/test/",
    augmix=True,
    random_color_jitter=False,
    toy=False,
)
# gftdm = GenericFinetuneDM()
dm.prepare_data()
dm.setup()
train_dataloader = dm.train_dataloader()

# print(train_dataloader.dataset.transform)

inputs, labels = next(iter(train_dataloader))
topil = transforms.ToPILImage()
imgs = [topil(img) for img in inputs]
imgs[1]

## Setup to create Multi-DataLoaders

In [ ]:
import logging

import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms


class MultiDataLoader:
    """
    Batch Sampler for a MultiDataset. Iterates in parallel over different batch samplers for each dataset.
    Yields batches [(x_1, y_1), ..., (x_s, y_s)] for s datasets.
    """

    def __init__(self, dataloaders):
        self._dataloaders = dataloaders
        self._n_batches = max([len(dl) for dl in dataloaders])
        self._init_iterators()

    def _init_iterators(self):
        self._iterators = [iter(dl) for dl in self._dataloaders]

    def _get_nexts(self):
        def _get_next_dl_batch(di, dl):
            try:
                batch = next(dl)
            except StopIteration:
                logging.debug(f"reinit loader {di} of type {type(dl)}")
                new_dl = iter(self._dataloaders[di])
                self._iterators[di] = new_dl
                batch = next(new_dl)
            return batch

        return [_get_next_dl_batch(di, dl) for di, dl in enumerate(self._iterators)]

    def __iter__(self):
        for _ in range(self._n_batches):
            yield self._get_nexts()
        self._init_iterators()

    def __len__(self):
        return self._n_batches


src_loader = dm.train_dataloader()
target_loader = dm.test_dataloader()

my_loader = MultiDataLoader([src_loader, target_loader])


def loop_through_my_loader(loader):
    """A use case of iterating through my_loader."""
    for i, batches in enumerate(loader):
        for j, b in enumerate(batches):
            data, target = b
            print(i, j, type(data), data.size(), type(target), target.size())
    print("num of batches: {}".format(i + 1))


for _ in range(2):
    loop_through_my_loader(my_loader)

print("len(my_loader):", len(my_loader))